In [1]:
# Imports
#---------
import sys
import pandas as pd
import numpy as np
import feather
import os
import gc
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
import matplotlib.ticker as ticker
from matplotlib.dates import DateFormatter
import matplotlib as mpl
from datetime import timezone

register_matplotlib_converters()

# File locations
#----------------
the_gases = '/home/tonyb/Gdrive/MinicondaProjects/oxaria/data/raw/2oxaria/json/gap_filling/'
pngs = '/home/tonyb/Gdrive/MinicondaProjects/oxaria/data/pngs/gap_filling/'


In [2]:
# Load the df 
#-------------
oxaria2_status = pd.read_feather(the_gases+'oxaria2_status_gf.ftr').set_index(['tag','rec']).sort_index()
oxaria2_status_lt = oxaria2_status.iloc[:,[12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]]
del oxaria2_status


In [3]:
# Checking out the data types 
#-----------------------------
print('\n'+'Loading Oxaria2 climate...\n')
oxaria2_status_lt.info()



Loading Oxaria2 climate...

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4254074 entries, ('scs-bgx-550', Timestamp('2020-03-09 10:31:48+0000', tz='UTC')) to ('scs-bgx-559', Timestamp('2021-02-28 23:59:37+0000', tz='UTC'))
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   val.up.load.av15   float32
 1   val.up.load.av1    float32
 2   val.up.load.av5    float32
 3   val.up.period      object 
 4   val.up.users       float32
 5   val.psu.prot-batt  float32
 6   val.psu.rst        float32
 7   val.psu.chg        float32
 8   val.psu.standby    object 
 9   val.psu.pwr-in     float32
 10  val.psu.host-3v3   float32
 11  val.psu.batt-flt   object 
 12  val.tz.name        object 
 13  val.tz.utc-offset  object 
 14  val.tmp.brd        float32
 15  name               object 
dtypes: float32(10), object(6)
memory usage: 564.2+ MB


In [4]:
# Some cleaning incase have missed anything
#-------------------------------------------
oxaria2_status_lt = oxaria2_status_lt[~oxaria2_status_lt.index.duplicated(keep='last')]
oxaria2_status_lt = oxaria2_status_lt.reset_index()
oxaria2_status_lt = oxaria2_status_lt.dropna(axis=0,subset=['tag']).set_index(['tag','rec'])
oxaria2_status_lt.sort_index()
print('\n Cleaning duplicates & NAs in index (if any)...\n')


 Cleaning duplicates & NAs in index (if any)...



In [5]:
# Get a list of unique tags
#---------------------------
tags = oxaria2_status_lt.index.get_level_values(0).unique()
print('\n Unique tags loaded...\n \n', tags)


 Unique tags loaded...
 
 Index(['scs-bgx-550', 'scs-bgx-551', 'scs-bgx-552', 'scs-bgx-553',
       'scs-bgx-554', 'scs-bgx-555', 'scs-bgx-556', 'scs-bgx-557',
       'scs-bgx-558', 'scs-bgx-559'],
      dtype='object', name='tag')


In [6]:
# Start dates for the devices
#-----------------------------
start_dates = ['2020-12-15T00:00:00','2020-07-20T00:00:00','2020-12-15T00:00:00','2020-07-01T00:00:00', \
               '2020-07-01T00:00:00','2020-12-15T00:00:00','2020-07-20T00:00:00','2020-07-01T00:00:00', \
               '2020-12-15T00:00:00','2020-12-15T00:00:00']
dates_list = [dt.datetime.fromisoformat(date).replace(tzinfo=timezone.utc) for date in start_dates]

# Gen dict of tags & start dates
#--------------------------------
dates_dict = dict(zip(tags,dates_list))
print('\n Applying variable start date filters of each sensor...\n \n'+str(dates_dict))


 Applying variable start date filters of each sensor...
 
{'scs-bgx-550': datetime.datetime(2020, 12, 15, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-551': datetime.datetime(2020, 7, 20, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-552': datetime.datetime(2020, 12, 15, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-553': datetime.datetime(2020, 7, 1, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-554': datetime.datetime(2020, 7, 1, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-555': datetime.datetime(2020, 12, 15, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-556': datetime.datetime(2020, 7, 20, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-557': datetime.datetime(2020, 7, 1, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-558': datetime.datetime(2020, 12, 15, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-559': datetime.datetime(2020, 12, 15, 0, 0, tzinfo=datetime.timezone.utc)}


In [10]:
# Select periods of stable operation from the df of all gases sesnor data
#-------------------------------------------------------------------------
print('\n Applying start date filters...\n')

tmp = []

for k,v in dates_dict.items():
    df = oxaria2_status_lt.query('tag == @k & rec >= @v')
    tmp.append(df)
tmpdf = pd.concat(tmp)
print(tmpdf.info())

# Save the df 
#-------------
print('\n Writing to  '+the_gases+'oxaria2_status_stable_356_2feb21.ftr\n')
tmpdf.reset_index().to_feather(the_gases+'oxaria2_status_stable_356_2feb21.ftr')
print('All done! \U0001F600')


 Applying start date filters...

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2000435 entries, ('scs-bgx-550', Timestamp('2020-12-23 14:59:33+0000', tz='UTC')) to ('scs-bgx-559', Timestamp('2021-02-28 23:59:37+0000', tz='UTC'))
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   val.up.load.av15   float32
 1   val.up.load.av1    float32
 2   val.up.load.av5    float32
 3   val.up.period      object 
 4   val.up.users       float32
 5   val.psu.prot-batt  float32
 6   val.psu.rst        float32
 7   val.psu.chg        float32
 8   val.psu.standby    object 
 9   val.psu.pwr-in     float32
 10  val.psu.host-3v3   float32
 11  val.psu.batt-flt   object 
 12  val.tz.name        object 
 13  val.tz.utc-offset  object 
 14  val.tmp.brd        float32
 15  name               object 
dtypes: float32(10), object(6)
memory usage: 270.6+ MB
None

 Writing to  /home/tonyb/Gdrive/MinicondaProjects/oxaria/data/raw/2oxaria/json/gap_filling/oxari